In [15]:
from pyevtk.hl import gridToVTK
from tqdm import tqdm
import numpy as np

In [6]:
def read (pre='eta_matrix_', t=100, index=0, N=512, filepath='/projects/DEIKE/jiarongw/multilayer/paraview/'):
    filename = filepath + pre + 't%g_' %t + 'l%g' %index
    f = np.fromfile(filename, dtype=np.float32)
    f = f.reshape(N+1,N+1); f = f[1:,1:]
    return f

def read_t (t=100, filepath='/projects/DEIKE/jiarongw/multilayer/paraview/', N=512, Nl=15):
    h_ensem = []; ux_ensem = []; uy_ensem = []; uz_ensem = []
    for l in range (0,Nl):
        h = read(pre='h_matrix_', t=t, index=l, N=N, filepath=filepath)
        ux = read(pre='ux_matrix_', t=t, index=l, N=N, filepath=filepath)
        uy = read(pre='uy_matrix_', t=t, index=l, N=N, filepath=filepath)
        uz = read(pre='uz_matrix_', t=t, index=l, N=N, filepath=filepath)
        h_ensem.append(h)
        ux_ensem.append(ux)
        uy_ensem.append(uy)
        uz_ensem.append(uz)

    h_ensem = np.array (h_ensem)
    ux_ensem = np.array (ux_ensem)
    uy_ensem = np.array (uy_ensem)
    uz_ensem = np.array (uz_ensem)
    
    return (h_ensem, ux_ensem, uy_ensem, uz_ensem)

In [13]:
""" convert the 3D array to vtk file for paraview. Need to specify L0 and H """

def array_to_vtk (h_ensem, ux_ensem, uy_ensem, uz_ensem, ichoice, 
                  filepath='/projects/DEIKE/jiarongw/multilayer/paraview/vtk/', 
                  L0=200, H=40, Nh=512, Nl=15):
    fieldnames = ['x', 'y', 'z', 'ux', 'uy', 'uz', 'f']

    x_vtk = np.zeros([Nh+1,Nh+1,Nl+1])
    y_vtk = np.zeros([Nh+1,Nh+1,Nl+1])
    z_vtk = np.zeros([Nh+1,Nh+1,Nl+1])
    f_vtk = np.zeros([Nh,Nh,Nl])
    ux_vtk = np.zeros([Nh,Nh,Nl])
    uy_vtk = np.zeros([Nh,Nh,Nl])
    uz_vtk = np.zeros([Nh,Nh,Nl])

    h_ensem_expand = np.zeros([Nl,Nh+1,Nh+1]) # Need to go from centered to grid, pad the array
    h_ensem_expand[:,:Nh,:Nh] = np.copy(h_ensem) # Need to go from centered to grid
    h_ensem_expand[:,Nh,:Nh] = np.copy(h_ensem[:,Nh-1,:Nh])
    h_ensem_expand[:,:Nh,Nh] = np.copy(h_ensem[:,:Nh,Nh-1])
    h_ensem_expand[:,Nh,Nh] = np.copy(h_ensem[:,Nh-1,Nh-1])
    h_ensem_expand = np.array(h_ensem_expand)

    xarray = np.linspace(-L0/2, L0/2, Nh+1, endpoint=True)
    yarray = np.linspace(-L0/2, L0/2, Nh+1, endpoint=True)

    for k in range(Nl+1):
        for i in range(Nh+1):
            for j in range(Nh+1):
                z_vtk[i,j,k] = np.sum(h_ensem_expand[:k,i,j]) - H
                x_vtk[i,j,k] = xarray[i]
                y_vtk[i,j,k] = yarray[j]

    for k in range(Nl):
        for i in range(Nh):
            for j in range(Nh):
                ux_vtk[i,j,k] = ux_ensem[k,i,j]
                uy_vtk[i,j,k] = uy_ensem[k,i,j]
                uz_vtk[i,j,k] = uz_ensem[k,i,j]
                if k == Nl-1: # surface layer
                    f_vtk[i,j,k] = 0
                else:
                    f_vtk[i,j,k] = 1
                    
    gridToVTK(filepath + "structured_%g" %ichoice, x_vtk, y_vtk, z_vtk, cellData = {"f": f_vtk, "ux": ux_vtk, "uy": uy_vtk, "uz": uz_vtk})
    return x_vtk, y_vtk, z_vtk

In [16]:
tchoice = 3.6
h_ensem, ux_ensem, uy_ensem, uz_ensem = read_t(t=tchoice, filepath='/projects/DEIKE/jiarongw/multilayer/revision/stokes_ml_1/field/', N=256, Nl=30)
x_vtk, y_vtk, z_vtk = array_to_vtk (h_ensem, ux_ensem, uy_ensem, uz_ensem, ichoice=3.6, 
                                    filepath='/projects/DEIKE/jiarongw/multilayer/revision/stokes_ml_1/vtk/',
                                    L0=1, H=0.5, Nh=256, Nl=30)
